# 07 — Daily report (end-to-end, using Universe Provider)

This notebook runs the full daily pipeline and exports a CSV.

New: tickers are loaded from the Universe Provider:
- `load_universe_from_package("mega")`
- optionally capped via `max_tickers` (like CLI `--top`)

Pipeline:
1) fetch OHLCV
2) build eligible universe
3) rank top N
4) compute entry signals
5) compute position sizing
6) generate final report

In [2]:
import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 80)

## 1) Load universe tickers

In [5]:
from swing_screener.data.universe import load_universe_from_package, UniverseConfig

tickers = load_universe_from_package("mega", UniverseConfig(max_tickers=50))
tickers[:15], len(tickers)

(['SPY',
  'QQQ',
  'DIA',
  'IWM',
  'XLK',
  'XLF',
  'XLE',
  'XLV',
  'XLY',
  'XLI',
  'XLP',
  'XLU',
  'XLB',
  'XLRE',
  'AAPL'],
 50)

## 2) Fetch market data

In [12]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2022-01-01"))
ohlcv.tail(), len(ohlcv)

(                  Open                                                                                                            \
                    SPY         QQQ         DIA         IWM         XLK        XLF        XLE         XLV         XLY         XLI   
 Date                                                                                                                               
 2026-01-12  690.679993  622.309998  492.151774  259.279999  145.190002  55.029999  46.750000  157.330002  123.870003  161.630005   
 2026-01-13  695.489990  627.270020  495.370611  262.410004  146.850006  55.150002  46.730000  157.649994  124.529999  163.880005   
 2026-01-14  691.000000  622.239990  490.482389  261.130005  145.500000  54.049999  47.250000  156.610001  123.389999  163.779999   
 2026-01-15  694.570007  626.599976  492.191773  264.059998  146.929993  54.220001  47.630001  157.309998  122.870003  165.070007   
 2026-01-16  693.659973  625.500000  494.500000  265.869995  146.7200

## 3) Build daily report (end-to-end)

In [14]:
from swing_screener.reporting.report import build_daily_report, export_report_csv, today_actions, ReportConfig
from swing_screener.risk.position_sizing import RiskConfig

cfg = ReportConfig(
    risk=RiskConfig(account_size=500, risk_pct=0.01, k_atr=2.0, max_position_pct=0.60),
    only_active_signals=False,
)

report = build_daily_report(ohlcv, cfg)
report

/Users/matteo.longo/projects/randomness/trading/swing_screener/src/swing_screener/reporting/report.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report["signal_order"] = report["signal"].map(order).fillna(99).astype(int)


,rank,score,last,atr14,atr_pct,mom_6m,mom_12m,rs_6m,trend_ok,dist_sma50_pct,dist_sma200_pct,signal,breakout_level,ma20_level,suggested_order_type,suggested_order_price,suggested_validity,execution_note,order_price_band_low,order_price_band_high
SLB,1,1.000000,46.730000,1.491428,3.191587,0.424852,0.186256,0.316425,True,20.235495,32.391926,none,46.970001,41.714001,SKIP,NaN,DAY,No actionable signal.,NaN,NaN
BAC,2,0.807143,52.970001,1.157857,2.185874,0.131476,0.150676,0.023048,True,-1.838204,10.364574,none,57.250000,55.248000,SKIP,NaN,DAY,No actionable signal.,NaN,NaN
XLB,3,0.571429,48.680000,0.667856,1.371932,0.089858,0.140843,-0.018570,True,8.447991,11.187666,none,48.959999,46.827251,SKIP,NaN,DAY,No actionable signal.,NaN,NaN
XLE,4,0.564286,47.689999,0.950714,1.993530,0.128594,0.064772,0.020166,True,5.716831,11.253457,none,48.060001,45.615700,SKIP,NaN,DAY,No actionable signal.,NaN,NaN
XLU,5,0.485714,43.389999,0.623571,1.437131,0.050969,0.162376,-0.057459,True,0.195515,4.069746,none,44.978031,42.756095,SKIP,NaN,DAY,No actionable signal.,NaN,NaN
PFE,6,0.328571,25.650000,0.452143,1.762740,0.085086,0.049044,-0.023342,True,1.395889,7.118769,none,26.430000,25.235000,SKIP,NaN,DAY,No actionable signal.,NaN,NaN
XLF,7,0.242857,54.439999,0.700000,1.285819,0.043207,0.115556,-0.065220,True,1.178557,5.040548,none,56.400002,55.177452,SKIP,NaN,DAY,No actionable signal.,NaN,NaN


## 4) Print a short action summary

In [15]:
print(today_actions(report, max_rows=10))

No tradable signals with current constraints (e.g. 500€). Today: no trade.


## 5) Export CSV

In [ ]:
export_path = export_report_csv(report, "out/daily_report.csv")
export_path


## Details: report merging and execution guidance
Report pipeline (`build_daily_report`):
- eligible universe -> ranking -> signals -> position sizing -> merged report.
- The report keeps ranked features and joins signal/plan columns when available.
- If a plan is not possible (e.g., too expensive), `signal` stays but plan fields are blank.
- `only_active_signals=True` filters to `both`, `breakout`, `pullback`.

Execution guidance (`add_execution_guidance`):
- Adds columns: `suggested_order_type`, `suggested_order_price`, `suggested_validity`, `execution_note`, `order_price_band_low/high`.
- Breakout not triggered -> BUY_STOP slightly above `breakout_level`.
- Breakout already triggered -> BUY_LIMIT pullback (if second-chance enabled).
- Pullback signal -> BUY_LIMIT near `ma20_level` with a small ATR band.



## Glossary (tickers and metrics)
- SPY: ETF tracking the S&P 500. We use it as the benchmark for Relative Strength (RS).
- SMA: Simple Moving Average. Used for trend filters and pullback signals.
- ATR: Average True Range. Measures typical daily range; used for stop distance and sizing.
- RS: Relative Strength = momentum vs benchmark (e.g., stock 6m return minus SPY 6m return).
- R (R-multiple): Risk unit where 1R = entry - stop. Used in backtests and trade management.
